In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import librosa

In [3]:
import math

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

 98% 1.19G/1.21G [00:23<00:00, 35.5MB/s]
100% 1.21G/1.21G [00:23<00:00, 54.4MB/s]


In [6]:
!unzip gtzan-dataset-music-genre-classification.zip

Archive:  gtzan-dataset-music-genre-classification.zip
  inflating: Data/features_30_sec.csv  
  inflating: Data/features_3_sec.csv  
  inflating: Data/genres_original/blues/blues.00000.wav  
  inflating: Data/genres_original/blues/blues.00001.wav  
  inflating: Data/genres_original/blues/blues.00002.wav  
  inflating: Data/genres_original/blues/blues.00003.wav  
  inflating: Data/genres_original/blues/blues.00004.wav  
  inflating: Data/genres_original/blues/blues.00005.wav  
  inflating: Data/genres_original/blues/blues.00006.wav  
  inflating: Data/genres_original/blues/blues.00007.wav  
  inflating: Data/genres_original/blues/blues.00008.wav  
  inflating: Data/genres_original/blues/blues.00009.wav  
  inflating: Data/genres_original/blues/blues.00010.wav  
  inflating: Data/genres_original/blues/blues.00011.wav  
  inflating: Data/genres_original/blues/blues.00012.wav  
  inflating: Data/genres_original/blues/blues.00013.wav  
  inflating: Data/genres_original/blues/blues.00014.wa

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
path = os.getcwd()
DATASET_PATH = os.path.join(path, 'Data/genres_original')
DATASET_PATH

'/content/Data/genres_original'

In [32]:
#path to sub_data
SUB_DATASET_PATH = os.path.join(os.getcwd(),'drive/MyDrive/Sub_Data/genres_original')
SUB_DATASET_PATH

'/content/drive/MyDrive/Sub_Data/genres_original'

In [33]:
def generating_headers(n_mfcc):
  headers = []
  for i in range(1,n_mfcc+1):
    headers.append(f'mfcc_{i}')
  return headers

In [34]:
def prepare_dataset(datapath, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    """
    Extracts mffcs feature from music dataset and saves them into a dataframe
    along with the genre associated with each audio file.
    """
    #Defining constants
    SAMPLE_RATE = 22050                                 #no of data samples taken per second
    TRACK_DURATION = 30                                 #duration of the track
    SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION    #total samples per track

    final_df = pd.DataFrame() # contains all the data samples
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment/hop_length)
    
    for dirpath, dirnames, filenames in os.walk(DATASET_PATH):
        #only process if the path is not the current working directory
        if dirpath is not DATASET_PATH:
            dirpath_components = dirpath.split('/')
            semantic_label = dirpath_components[-1]
            print(f'\nProcessing: {semantic_label}')

            single_file_df = pd.DataFrame()
            #processing each file within a specific genre
            for file in filenames:
                file_path = os.path.join(dirpath, file)
                label = file_path.split('/')[-2]

                try:
                    #load the audio file
                    signal, sr = librosa.load(file_path, sr = SAMPLE_RATE)

                    #process all segments extracting mfcc and storing data
                    for segment in range(num_segments):
                        #setting the start and finish index of a sample for each segment
                        start_sample = num_samples_per_segment * segment
                        finish_sample = start_sample + num_samples_per_segment

                        #extracting mfcc features for a single segment
                        mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                                   sr = sr,
                                                   n_mfcc = n_mfcc,
                                                   n_fft = n_fft,
                                                   hop_length = hop_length)
                        mfcc = mfcc.T

                        #store mfcc feature for a single segment only if the number of vectors
                        #for each segment contains expected number of vectors
                        if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                            segment_df = pd.DataFrame(data = mfcc,
                                                     columns= generating_headers(n_mfcc),
                                                     index = range(len(mfcc)))
                            segment_df['Labels'] = [label] * mfcc.shape[0]
                            #nparray.append(mfcc)
                            #concatenating segment dataframe into a single file df
                            single_file_df = pd.concat([single_file_df, segment_df],
                                                      axis=0,
                                                      sort=False,
                                                      ignore_index = True)                        
                except Exception as e:
                    print(f"Error Message: {e}")
            final_df = pd.concat([final_df, single_file_df],
                         axis=0,
                         sort = False,
                         ignore_index = True)
        print('Done with preparing dataset')
    return final_df


In [35]:
df = prepare_dataset(datapath=DATASET_PATH, num_segments=10)

Done with preparing dataset

Processing: disco
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset

Processing: metal
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset

Processing: classical
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset

Processing: blues
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset

Processing: pop
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset

Processing: rock
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset

Processing: country
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset

Processing: hiphop
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset

Processing: jazz


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error Message: 
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset

Processing: reggae
Done with preparing dataset

Processing: .ipynb_checkpoints
Done with preparing dataset


In [36]:
df

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,Labels
0,27.555986,88.491943,-20.645388,18.461662,-22.876869,26.373840,-7.842380,5.040419,-10.529922,30.722309,8.053223,25.249620,-3.851202,disco
1,21.160398,99.239639,-33.311604,24.493477,-17.089005,25.532211,-5.425315,11.550676,-12.097195,27.441135,0.058880,19.927771,-11.912487,disco
2,-13.337629,112.291656,-48.217140,38.652039,-11.287874,30.100903,-4.544308,16.824894,-15.798969,27.857822,-13.360273,20.098824,-20.572792,disco
3,-40.871708,116.767059,-47.124802,49.234226,-17.217890,37.331268,-0.842096,16.252968,-20.979574,27.505018,-24.680790,19.476368,-18.812157,disco
4,-67.497528,117.398331,-47.275208,47.681198,-16.081495,43.743324,13.473402,14.318071,-27.892868,20.559271,-31.776064,17.269453,-14.578279,disco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233175,-101.685333,23.818932,0.495472,30.757507,17.166796,16.255051,18.272770,9.566826,11.331477,4.932149,9.956822,12.857517,7.169489,reggae
1233176,-136.458862,21.216267,-1.512686,37.810467,22.128681,20.463276,16.296101,12.380598,18.769466,8.667844,15.075156,15.303005,7.611032,reggae
1233177,-167.509811,25.255136,0.489490,48.138466,27.790737,23.130043,17.280693,13.791199,17.217871,7.926507,13.045273,12.954222,9.367888,reggae
1233178,-152.141953,51.688583,-4.779652,27.629368,14.154837,21.258694,11.772314,17.563957,16.069324,20.217516,16.887005,8.754175,-7.446566,reggae


In [37]:
df['Labels'].unique()

array(['disco', 'metal', 'classical', 'blues', 'pop', 'rock', 'country',
       'hiphop', 'jazz', 'reggae'], dtype=object)

In [38]:
len(df['Labels'].unique())

10

# Saving Dataframe in a pickle file

In [39]:
import pickle

In [40]:
import os

In [41]:
os.getcwd()

'/content'

In [42]:
os.listdir(os.getcwd())

['.config',
 'drive',
 'kaggle.json',
 'gtzan-dataset-music-genre-classification.zip',
 '.ipynb_checkpoints',
 'Data',
 'sample_data']

In [43]:
df.to_pickle('data_13_G.pickle')

In [44]:
df.to_csv('data_13_G.csv')

In [45]:
DATASET_PATH

'/content/Data/genres_original'

Testing Encoding

In [46]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [47]:
encoder = LabelEncoder()
targets = encoder.fit_transform(df['Labels'].unique())
targets

array([3, 6, 1, 0, 7, 9, 2, 4, 5, 8])

In [48]:
labels = encoder.inverse_transform(targets)

In [49]:
labels

array(['disco', 'metal', 'classical', 'blues', 'pop', 'rock', 'country',
       'hiphop', 'jazz', 'reggae'], dtype=object)